In [ ]:
# import packages required
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector

# Creating a connection to mysql to import data into database
try:
    mydb = mysql.connector.connect(
        host = '127.0.0.1',
        user = 'root',
        password = 'ShahulSqL2024'
    )
    print("Connection Established")
    cursor = mydb.cursor()
    cursor.execute("create database if not exists redbus")
    mydb.commit()
    print("Database created successfully")
    cursor.execute("use redbus")
    cursor.execute("drop table if exists bus_routes")

except mysql.connector.Error as e:
    print(f"An error occured {e}")

host = '127.0.0.1'
db = 'redbus'
user = 'root'
pw = 'ShahulSqL2024'

engine = create_engine(f"mysql+pymysql://{user}:{pw}@{host}/{db}")

# inititate browser
driver = webdriver.Chrome()

# initiate action chains
actions = ActionChains(driver)


transport_state = ['Andhra Pradesh','Kerala','Telangana','Karnataka','Rajasthan','South Bengal','Haryana','Assam','Uttar Pradesh','West Bengal','Chandigarh','Punjab']
transport_info = [
    'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile',
    'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
    'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile'
    ]

# scroll pages to the bottom to load all the data
def is_at_end_of_page(driver):
    # Execute JavaScript to get the scroll position, document height, and viewport height
    scroll_position = driver.execute_script("return window.scrollY;")
    page_height = driver.execute_script("return document.documentElement.scrollHeight;")
    viewport_height = driver.execute_script("return window.innerHeight;")
    
    # Check if we've reached the bottom of the page
    return scroll_position + viewport_height >= page_height


for j,k in zip(transport_info,transport_state):
    # opening web page
    driver.get(j);
    driver.maximize_window()
    time.sleep(3)

    # declaring empty list to store values
    route_name, route_link = [], []
    rte_name, rte_link = [], []
    bus_name, bus_type, departing_time, duration, reaching_time = [], [], [], [], []
    star_rating, price, seat_availability, state = [], [], [], []

    # identifying the number of pages found in each web page
    no_of_pages = len(driver.find_elements(By.CLASS_NAME, 'DC_117_pageTabs '))

    # looping through the pages
    for n in range(1,no_of_pages+1):
        actions.move_to_element(driver.find_element(By.XPATH, "//*[@id='root']/div/div[4]/div[12]/div[text()='" + str(n) + "']")).click()
        actions.perform()
        temp_route = driver.find_elements(By.XPATH, "//a[@class='route']")
        
        # looping through the selenium object to extract data into the list
        for i in temp_route:
            rte_name.append(i.text)
            rte_link.append(i.get_attribute('href'))

    # looping through the extracted 
    for r,s in zip(rte_name,rte_link):
        
        driver.get(s);
        time.sleep(10)

        check = len(driver.find_elements(By.XPATH,"//h3[text()='Oops! No buses found.']"))

        if check > 0:
            actions.move_to_element(driver.find_element(By.XPATH,"//*[@id='fixer']/div/div/div[1]/span[@class='next']")).click()
            actions.perform()
            time.sleep(5)

        # identify the number of expandable options within the webpage
        no_of_clicks = driver.find_elements(By.XPATH,"//*[@class='button']")

        clicks = len(no_of_clicks)

        if clicks <= 1:
            for t in no_of_clicks:
                t.click()
                time.sleep(3)
        else:
            clicks = len(no_of_clicks)-1
            for t in range(clicks,-1,-1):
                no_of_clicks[t].click()
                time.sleep(5)

        # Scroll down and check if we are at the end
        while not is_at_end_of_page(driver):
            # Scroll down by sending PAGE_DOWN or by using JavaScript
            driver.find_element('tag name', 'body').send_keys(Keys.PAGE_DOWN)

        list_items = driver.find_elements(By.XPATH, "//*[@class='row-sec clearfix']")

        for a in list_items:
            state.append(k)
            route_name.append(r)
            route_link.append(s)
            bus_name.append(a.find_element(By.XPATH, "//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[1]/div[1]").text)
            bus_type.append(a.find_element(By.XPATH, "//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[1]/div[2]").text)
            departing_time.append(a.find_element(By.XPATH,"//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[2]/div[1]").text)
            duration.append(a.find_element(By.XPATH,"//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[3]/div").text)
            reaching_time.append(a.find_element(By.XPATH,"//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[4]/div[1]").text)
            try:
                element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[5]/div[1]/div/span"))
                )
                star_rating.append(element.text)
            except TimeoutException:
                star_rating.append(0)
            price.append(a.find_element(By.XPATH,"//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[6]/div/div/span").text)
            seat_availability.append((a.find_element(By.XPATH,"//*[@id='" + a.get_attribute('id') + "']/div/div[1]/div[1]/div[7]/div").text).split(' ')[0])
        
    col_name = ['state','bus_route_name', 'bus_route_link', 'bus_name', 'bus_type', 'departing_time', 'duration', 'reaching_time', 'star_rating', 'price', 'seat_availability']
    df = pd.DataFrame(list(zip(state,route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability)),columns=col_name)

    df.to_sql('bus_routes',engine,if_exists='append',index=False)
    df.to_csv(f"{k}.csv",index=False)
    print(f"{k} is successfully extracted")

cursor.execute('Alter table bus_routes add column id int auto_increment primary key')
cursor.execute('Alter table bus_routes modify column departing_time time')
cursor.execute('Alter table bus_routes modify column reaching_time time')
cursor.execute('Alter table bus_routes modify column star_rating float')
cursor.execute('Alter table bus_routes modify column price decimal')
cursor.execute('Alter table bus_routes modify column seat_availability int')

print('Data Extracted Successfully!')

driver.quit()

Connection Established
Database created successfully


KeyboardInterrupt: 

In [2]:
for i in bus_name:
    print(i)

APSRTC - 3576
APSRTC - 9363
APSRTC - 24291
APSRTC - 3578
APSRTC - 94285
APSRTC - 3722
APSRTC - 3859
APSRTC - 3356
APSRTC - 3546
APSRTC - 3500
APSRTC - 3376
APSRTC - 2561
APSRTC - 3358
APSRTC - 2674
APSRTC - 3366
APSRTC - 3360
APSRTC - 2898
APSRTC - 4916
APSRTC - 2563
APSRTC - 2680
APSRTC - 2798
APSRTC - 2688
APSRTC - 3622
APSRTC - 4972
APSRTC - 2549
APSRTC - 2573
APSRTC - 4894
APSRTC - 24262
APSRTC - 3971
APSRTC - 4898
APSRTC - 2786
APSRTC - 3861
APSRTC - 2678
APSRTC - 4880
APSRTC - 3697
APSRTC - 2876
APSRTC - 3977
APSRTC - 4351
APSRTC - 38202
APSRTC - 37801
APSRTC - 3544
APSRTC - 4896
APSRTC - 4179
APSRTC - 3969
APSRTC - 4900
APSRTC - 3612
APSRTC - 3962
APSRTC - 95703
APSRTC - 4795
APSRTC - 4043
APSRTC - 4353
APSRTC - 4823
APSRTC - 3883
APSRTC - 35212
APSRTC - 35064
APSRTC - 35172
APSRTC - 4920
APSRTC - 2676
APSRTC - 9652
APSRTC - 4936
APSRTC - 4815
APSRTC - 9655
APSRTC - 4996
APSRTC - 35136
APSRTC - 3570
APSRTC - 3618
APSRTC - 3917
APSRTC - 4789
APSRTC - 3691
APSRTC - 4340
APSRTC - 4

In [4]:
from sqlalchemy import create_engine
import mysql.connector

# Creating a connection to mysql to import data into database
try:
    mydb = mysql.connector.connect(
        host = '127.0.0.1',
        user = 'root',
        password = 'ShahulSqL2024'
    )
    print("Connection Established")
    cursor = mydb.cursor()
    cursor.execute("create database if not exists redbus")
    mydb.commit()
    print("Database created successfully")
    cursor.execute("use redbus")
    cursor.execute("drop table if exists bus_routes")

except mysql.connector.Error as e:
    print(f"An error occured {e}")

host = '127.0.0.1'
db = 'redbus'
user = 'root'
pw = 'ShahulSqL2024'

engine = create_engine(f"mysql+pymysql://{user}:{pw}@{host}/{db}")

file_list = ['C:/Users/Shahul_Desktop/hello/Data Extracted/Andhra Pradesh.csv','C:/Users/Shahul_Desktop/hello/Data Extracted/Assam.csv',
             'C:/Users/Shahul_Desktop/hello/Data Extracted/Chandigarh.csv','C:/Users/Shahul_Desktop/hello/Data Extracted/Haryana.csv',
             'C:/Users/Shahul_Desktop/hello/Data Extracted/Karnataka.csv','C:/Users/Shahul_Desktop/hello/Data Extracted/Kerala.csv',
             'C:/Users/Shahul_Desktop/hello/Data Extracted/Punjab.csv','C:/Users/Shahul_Desktop/hello/Data Extracted/Rajasthan.csv',
             'C:/Users/Shahul_Desktop/hello/Data Extracted/South Bengal.csv','C:/Users/Shahul_Desktop/hello/Data Extracted/Telangana.csv',
             'C:/Users/Shahul_Desktop/hello/Data Extracted/Uttar Pradesh.csv','C:/Users/Shahul_Desktop/hello/Data Extracted/West Bengal.csv'
]

for i in file_list:
    df = pd.DataFrame(pd.read_csv(i))
    df.to_sql('bus_routes',engine,if_exists='append',index=False)

cursor.execute('Alter table bus_routes add column id int auto_increment primary key')
cursor.execute('Alter table bus_routes modify column departing_time time')
cursor.execute('Alter table bus_routes modify column reaching_time time')
cursor.execute('Alter table bus_routes modify column star_rating float')
cursor.execute('Alter table bus_routes modify column price decimal')
cursor.execute('Alter table bus_routes modify column seat_availability int')

Connection Established
Database created successfully


In [ ]:
import pandas as pd

df = pd.read_csv("C:/Users/Shahul_Desktop/hello/Data Extracted/Andhra Pradesh.csv")
df['departing_time'] = pd.to_datetime(df['departing_time']).dt.time
df['reaching_time'] = pd.to_datetime(df['reaching_time']).dt.time
df

C:\Users\Shahul_Desktop\AppData\Local\Temp\ipykernel_16920\3241721106.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['departing_time'] = pd.to_datetime(df['departing_time']).dt.time
C:\Users\Shahul_Desktop\AppData\Local\Temp\ipykernel_16920\3241721106.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['reaching_time'] = pd.to_datetime(df['reaching_time']).dt.time


,state,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35086,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),11:15:00,05h 45m,17:00:00,3.8,683.0,40
1,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3580,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),12:15:00,05h 45m,18:00:00,3.8,683.0,38
2,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 24256,INDRA(A.C. Seater),12:30:00,05h 30m,18:00:00,4.2,528.0,34
3,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3576,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),13:15:00,06h 15m,19:30:00,3.8,683.0,36
4,Andhra Pradesh,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35154,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:40:00,06h 40m,20:20:00,3.7,444.0,32
...,...,...,...,...,...,...,...,...,...,...,...
5154,Andhra Pradesh,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Elegance Tours And Travels Pvt Ltd,Bharat Benz A/C Sleeper (2+1),22:30:00,05h 20m,03:50:00,1.9,3000.0,36
5155,Andhra Pradesh,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Elegance Tours And Travels Pvt Ltd,Bharat Benz A/C Sleeper (2+1),22:30:00,05h 20m,03:50:00,1.9,3000.0,25
5156,Andhra Pradesh,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,KKaveri Travels,Non A/C Seater / Sleeper (2+1),20:30:00,05h 30m,02:00:00,1.6,1290.0,28
5157,Andhra Pradesh,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,KKaveri Travels,Volvo Multi-Axle A/C Sleeper (2+1),20:45:00,07h 15m,04:00:00,1.6,1790.0,30


In [19]:
import streamlit as st
import mysql.connector
import pandas as pd
import datetime


# Function to connect to MySQL and fetch data
def fetch_data(inp):
    try:
        # Establish the MySQL connection
        mydb = mysql.connector.connect(
            host='127.0.0.1',  # MySQL host
            user='root',       # Your MySQL username
            password='ShahulSqL2024',  # Your MySQL password
            database='redbus'  # The database you want to connect to
        )
        
        # Creating a cursor object to interact with the database
        cursor = mydb.cursor()

        # Execute the query
        cursor.execute(inp)  # Change 'your_table' to your table name

        # Fetch all rows from the executed query
        rows = cursor.fetchall()

        # Get column names from cursor.description
        columns = [desc[0] for desc in cursor.description]

        # Create a DataFrame using Pandas
        df = pd.DataFrame(rows, columns=columns)

        # Return the DataFrame
        return df
    
    except mysql.connector.Error as e:
        st.write(f"An error occurred: {e}")
        return None

    finally:
        # Ensure to close the connection after operation
        if mydb.is_connected():
            cursor.close()
            mydb.close()

# Fetch data from MySQL and display it in Streamlit
query = 'SELECT * FROM bus_routes order by id asc'
data = fetch_data(query)
data['departing_time'] = pd.to_timedelta(data['departing_time'])
data['depart'] = data['departing_time'].dt.components.apply(lambda x: f"{x['hours']:02}:{x['minutes']:02}:{x['seconds']:02}",axis=1)
#data['reaching_time'] = pd.to_datetime(data['reaching_time']).dt.time
print(data['depart'])

0        11:15:00
1        12:15:00
2        12:30:00
3        13:15:00
4        13:40:00
           ...   
16778    16:00:00
16779    12:40:00
16780    14:15:00
16781    21:50:00
16782    15:15:00
Name: depart, Length: 16783, dtype: object
